In [17]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# from HelperFunctions import minibatch 
%reload_ext autoreload
%autoreload 2
from HelperFunctions import minibatch, dummify_columns, undummify, feature_standardize, label_encode_column, columns_of_type

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import metrics

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [18]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, AdaBoostRegressor
randomForest = RandomForestRegressor()
gbm = GradientBoostingRegressor()
abr = AdaBoostRegressor()
from xgboost import XGBRegressor
xgb = XGBRegressor()
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()

In [24]:
df=pd.read_csv('down_sampled_df_v2.csv')


In [25]:
drop_list=['id','sub_grade','issue_d','zip_code','addr_state','RANDOM']
df.drop(drop_list,axis=1,inplace=True)

In [26]:
x_train=df.drop(['loan_status','return_rate'],axis=1)
y_train=df.return_rate

In [27]:
cat_list=columns_of_type(x_train,'string')
cont_list=columns_of_type(x_train,'number')

# Label Encode Dataframe for random forest

In [28]:
label_encoded_df=label_encode_column(x_train,cat_list)
label_encoded_df.sample(10)

,funded_amnt,term,int_rate,emp_length,annual_inc,dti,delinq_2yrs,credit_age_years,mths_since_last_delinq,mths_since_last_derog_record,derog_records,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,collection_amt,install_util,all_util,rev_credit,inq_last_12m,chargeoff_within_12_mths,delinq_amnt,mths_since_RecentAcc_opened,all_accs_120days+_PastDue_ever,accs_90days+_PastDue_24m,accs_opened_past_12m,pct_acc_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_credit,install_credit,fico,Outstanding_mortgage_debt,revol_frac,install_frac,mort_frac,card_frac,active_card_frac,active_revol_frac,active_install_frac,open_revol_frac,good_acc_frac,loan_duration,grade,home_ownership,verification_status,purpose,initial_list_status,application_type
44044,20000.0,60,17.88,1,82000.0,18.31,0.0,23.413889,63.0,-999.0,0.0,94.7,22.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0,-999.0,-999.0,702.0,-999.0,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,60.900000,4,1,2,9,0,0
13446,12000.0,36,13.49,1,64000.0,9.98,0.0,20.208333,-999.0,-999.0,0.0,81.4,9.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0,-999.0,-999.0,717.0,-999.0,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,13.200000,2,5,1,2,0,0
27868,10000.0,36,14.85,3,60000.0,22.58,0.0,11.072222,-999.0,81.0,1.0,72.0,20.0,0.0,30.0,0.0,0.0,84.0,72.0,10390.0,4.0,0.0,0.0,2.0,0.0,0.0,5.0,55.0,1.0,0.0,47907.0,37517.0,662.0,0.0,0.750000,0.200000,0.050000,0.533333,0.375000,0.466667,0.500000,0.466667,0.450000,17.233333,2,5,0,2,1,0
45907,23150.0,60,25.49,1,55169.0,24.37,0.0,20.716667,44.0,-999.0,0.0,35.0,52.0,0.0,44.0,0.0,0.0,81.0,54.0,43600.0,1.0,0.0,0.0,14.0,1.0,0.0,0.0,97.8,0.0,0.0,242150.0,198550.0,677.0,0.0,0.538462,0.442308,0.019231,0.214286,0.833333,0.571429,0.173913,0.785714,0.480769,1.966667,4,5,1,1,1,0
21574,20000.0,60,12.29,1,69700.0,12.80,0.0,18.850000,-999.0,-999.0,0.0,57.6,22.0,0.0,-999.0,0.0,0.0,-999.0,-999.0,18700.0,-999.0,0.0,0.0,34.0,0.0,0.0,0.0,100.0,0.0,0.0,46070.0,16460.0,697.0,0.0,0.818182,0.090909,0.000000,0.444444,0.625000,0.444444,-999.000000,0.666667,0.681818,28.466667,2,5,1,1,1,0
39664,36000.0,60,21.45,1,96000.0,29.13,1.0,26.713889,8.0,-999.0,0.0,78.1,19.0,0.0,-999.0,0.0,0.0,49.0,62.0,47700.0,5.0,0.0,0.0,34.0,0.0,0.0,0.0,88.9,0.0,0.0,112845.0,49487.0,687.0,0.0,0.578947,0.210526,0.157895,0.818182,0.333333,0.454545,0.500000,0.636364,0.526316,12.166667,3,5,0,2,1,0
48212,3850.0,36,23.43,1,50000.0,27.72,0.0,13.691667,-999.0,-999.0,0.0,81.0,26.0,0.0,-999.0,0.0,0.0,-999.0,-999.0,19600.0,-999.0,0.0,0.0,7.0,0.0,0.0,2.0,100.0,0.0,0.0,54959.0,35359.0,672.0,0.0,0.884615,0.038462,0.000000,0.565217,0.538462,0.478261,-999.000000,0.608696,0.576923,36.533333,5,5,0,12,1,0
22185,21600.0,60,16.29,8,60000.0,21.72,0.0,12.772222,73.0,-999.0,0.0,56.2,36.0,0.0,77.0,0.0,0.0,-999.0,-999.0,30000.0,-999.0,0.0,0.0,6.0,1.0,0.0,3.0,83.0,0.0,0.0,61955.0,31955.0,697.0,0.0,0.694444,0.250000,0.055556,0.640000,0.250000,0.280000,-999.000000,0.400000,0.361111,29.333333,2,1,2,2,0,0
37427,12000.0,36,14.09,2,42000.0,11.69,0.0,10.058333,-999.0,-999.0,0.0,82.7,9.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0,-999.0,-999.0,667.0,-999.0,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,31.433333,3,5,0,2,0,0
45300,22250.0,60,21.98,4,50000.0,27.12,0.0,7.444444,-999.0,-999.0,0.0,8.0,16.0,0.0,-999.0,0.0,329.0,-999.0,-999.0,8300.0,-999.0,0.0,0.0,3.0,0.0,0.0,2.0,100.0,0.0,0.0,108839.0,62039.0,697.0,24668.0,0.187500,0.687500,0.062500,0.333333,1.000000,0.333333,-999.000000,0.666667,0.500000,50.666667,4,1,2,5,0,0


# Standardize Features

In [29]:
final_df=label_encoded_df.apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)

# Random Forest

In [ ]:
def error_model(params):
    regressor = RandomForestRegressor(n_estimators=100,**params)
    return -1*cross_val_score(regressor, label_encoded_df,
                              y_train,cv=5,n_jobs=-1,scoring='neg_mean_squared_error').mean()

max_depth_range=range(3,7)
max_features_range=range(5,8)

param_space = {
    'max_depth': hp.choice('max_depth',max_depth_range),
    'max_features': hp.choice('max_features', max_features_range)}

best = error_model({'max_depth':max_depth_range[0],'max_features':max_features_range[0]})
def f(params):
    global best
    error_score = error_model(params)
    if error_score < best:
        best = error_score
    print ('new best:', best, params)
    return {'loss': error_score, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, param_space, algo=tpe.suggest, max_evals=15, trials=trials)
print ('best:')
print (best)